# Particle Fluid

Modeling fluid motion by simulating a set of particles. 

Author: Andres Ibarra, written as part of 22F Leave Term Research

Credit to Professor Bo Zhu, on whose Dartmouth COSC89 course notes the following code is based

In [1]:
import taichi as ti
import taichi.math as tm
import math

ti.init()

[Taichi] version 1.2.1, llvm 10.0.0, commit 12ab828a, osx, python 3.9.7
[I 11/21/22 00:02:16.486 1677761] [shell.py:_shell_pop_print@33] Graphical python shell detected, using wrapped sys.stdout
[Taichi] Starting on arch=x64


In [2]:
### Global variables ###
dt = 2e-3
max_particles = 400
init_num_particles = 300
num_particles = ti.field(dtype=ti.i32, shape=())
substeps = 10
pressure_coef = 1e1
density_0 = 20.0
viscosity_coef = 80e1
gravity = tm.vec2(0, -1)
max_num_particles_per_cell = 100
max_num_neighbors = 100
screen_size = 800
ks = 1e2
kd = 0.5e1

particle_block_size = 16 # particles to add at a time with a mouse click
new_particle_radius = 5
new_particle_mass = 3

particle_radius = 5
radius_to_screen = particle_radius / screen_size
h_raw = 15 # in terms of pts
h = h_raw/ screen_size # fraction of the screen that it represents
bowl_r = 500
c = tm.vec2(0.5, 0.7) # center of the bowl

num_voxels_per_side = int(screen_size / h_raw) # assuming a square screen
num_voxels = int(pow(screen_size / h_raw, 2))

pos = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
vel = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
force = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
r = ti.field(dtype=ti.f32, shape=max_particles)
m = ti.field(dtype=ti.f32, shape=max_particles)
density = ti.field(dtype=ti.f32, shape=max_particles)
pressure = ti.field(dtype=ti.f32, shape=max_particles)
viscosity = ti.field(dtype=ti.f32, shape=max_particles)
neighbors = ti.field(dtype=ti.i32, shape=(max_particles, max_num_neighbors)) 
voxels = ti.field(dtype=ti.i32, shape=(num_voxels, max_num_particles_per_cell)) 

particle_num_neighbors = ti.field(dtype=ti.i32, shape=max_particles)
grid_num_particles = ti.field(dtype=ti.i32, shape=num_voxels)

debug = False

In [3]:
### Spiky Kernel Functions ###
@ti.func
def w_spiky(xji) -> ti.f32:
    """Returns the value of the spiky kernel, with xji as the displacement."""
    valToReturn = float(0)
    r = xji.norm()
    if (r >= 0 and r <= h):
        valToReturn = 15.0 / (math.pi * ti.pow(h, 6)) * ti.pow(h - r, 3)
    return valToReturn

@ti.func
def gradient_w_spiky(v) -> tm.vec2: 
    valToReturn = tm.vec2(0,0)
    r = v.norm() 
    if (r > 0 and r <= h):
        valToReturn = -45.0 / (math.pi * ti.pow(h, 6)) * ti.pow (h - r, 2) * v / r
    return valToReturn

@ti.func
def laplacian_w_spiky(v) -> ti.f32:
    valToReturn = 0.0
    r = v.norm() 
    if (r > 0 and r <= h):
        valToReturn = -90.0 / (math.pi * ti.pow(h, 6)) * (h - r) * (h - 2*r) / r
    return valToReturn


In [4]:
## Viscosity Kernel Functions ###
@ti.func
def w_vis(xji) -> ti.f32:
    """Returns the value of the viscosity kernel, with xji as displacement."""
    valToReturn = 0.0
    r = xji.norm()
    if (r > 0 and r <= h):
        valToReturn = 15.0 / (2 * math.pi * ti.pow(h,3)) * ((- ti.pow(r, 3) / (2 * ti.pow(h, 3)) + ti.pow(r, 2) / ti.pow(h, 2) + h / (2 * r) - 1))
    return valToReturn

@ti.func
def gradient_w_vis(v) -> tm.vec2:
    valToReturn = tm.vec2(0,0)
    r = v.norm()
    if (r > 0 and r <= h):
        valToReturn = 15.0 / (2 * math.pi * ti.pow(h, 3)) * (-3 * r / (2 * ti.pow(h, 3)) + 2 / ti.pow(h, 2) - h/ (2 * ti.pow(r, 3))) * v
    return valToReturn

@ti.func
def laplacian_w_vis(v) -> ti.f32:
    valToReturn = 0.0
    r = v.norm()
    if (r >= 0 and r <= h):
        valToReturn = 45.0 / (math.pi * ti.pow(h, 6)) * (h - r)
    return valToReturn


In [5]:
### Spatial Hashing Functions ###

@ti.func
def get_voxel_num_from_idx(idx:ti.i32) -> ti.i32:
    """Returns the voxel number a point belongs to given its index."""
    voxelX = int(pos[idx][0] / h)
    voxelY = int(pos[idx][1] / h)
    voxel_num = voxelY * num_voxels_per_side + voxelX
    return voxel_num

@ti.func
def get_voxel_num(voxelX:ti.i32, voxelY:ti.i32) -> ti.i32:
    """Returns the voxel number given the voxelX and voxelY"""
    return int(voxelY * num_voxels_per_side + voxelX)

@ti.func
def get_voxel_x(idx:ti.i32) -> ti.i32:
    return int(pos[idx][0] / h)

@ti.func
def get_voxel_y(idx:ti.i32) -> ti.i32:
    return int(pos[idx][1] / h)

@ti.func
def clear_voxels():
    """Resets all voxel index lists to be 1"""
    for i in range(num_voxels):
        grid_num_particles[i] = 0
        for j in range(max_num_particles_per_cell):
            voxels[i, j] = -1

@ti.func
def add_point(idx:ti.i32):
    """Adds points with index idx to the relevant voxel"""
    voxel_num = get_voxel_num_from_idx(idx)
    
    count = grid_num_particles[voxel_num]
    voxels[voxel_num, count] = idx
    grid_num_particles[voxel_num] += 1

@ti.func
def update_voxels():
    """Spatial hashing function performed at each timestep."""
    clear_voxels()
    for idx in range(num_particles[None]):
        add_point(idx)

In [6]:
### Simulator Functions ###

@ti.func
def clear_force():
    for idx in range(num_particles[None]):
        force[idx] = tm.vec2(0, 0)

@ti.func
def clear_neighbors():
    """Reset all neighbors index lists to be -1."""
    for i in range(num_particles[None]):
        particle_num_neighbors[i] = 0
        for j in range(max_num_neighbors):
            neighbors[i, j] = -1
            
@ti.func
def update_neighbors_helper(idx:ti.i32, voxelX:ti.i32, voxelY:ti.i32):
    """Adds all particles in voxel (voxelX, voxelY) to the neighbors list of point idx."""
    voxel_num = get_voxel_num(voxelX, voxelY)
    count = particle_num_neighbors[idx] # where in idx's neighbors array we're adding
    voxCounter = 0 # counter for iterating through this voxel's particle ids
    while voxels[voxel_num, voxCounter] != -1 and count < max_num_neighbors:
        neighbors[idx, count] = voxels[voxel_num, voxCounter] 
        particle_num_neighbors[idx] += 1
        count = count+1
        voxCounter = voxCounter+1
        
@ti.func
def update_neighbors():
    clear_neighbors()
    for idx in range(num_particles[None]):
        # look at 9 cells surrounding
        voxelX = get_voxel_x(idx)
        voxelY = get_voxel_y(idx)
        # upper row
        if voxelY > 0:
            if voxelX > 0: # upper left
                update_neighbors_helper(idx, voxelX-1, voxelY+1)
            # up
            update_neighbors_helper(idx, voxelX, voxelY+1)
            if voxelX < num_voxels_per_side: # upper right
                update_neighbors_helper(idx, voxelX+1, voxelY+1)
        # middle row
        if voxelX > 0: # left
            update_neighbors_helper(idx, voxelX-1, voxelY)
        # center
        update_neighbors_helper(idx, voxelX, voxelY)
        if voxelX < num_voxels_per_side: # right
            update_neighbors_helper(idx, voxelX+1, voxelY)
        
        # lower row
        if voxelY < num_voxels_per_side:
            if voxelX > 0: # lower left
                update_neighbors_helper(idx, voxelX-1, voxelY-1)
            # down
            update_neighbors_helper(idx, voxelX, voxelY-1)
            if voxelX < num_voxels_per_side: # lower right
                update_neighbors_helper(idx, voxelX+1, voxelY-1)

@ti.func
def update_density():
    for idx in range(num_particles[None]):
        d = 0.0
        for nbr in range(particle_num_neighbors[idx]):
            nbr_idx = neighbors[idx, nbr]
            xji = pos[idx] - pos[nbr_idx]
            d += m[nbr_idx] * w_spiky(xji)
        density[idx] = d

@ti.func
def update_pressure():
    for idx in range(num_particles[None]):
        pressure[idx] = pressure_coef * (density[idx] - density_0)

@ti.func
def update_pressure_force():
    for idx in range(num_particles[None]):
        currForce = tm.vec2(0,0)
        for nbr in range(particle_num_neighbors[idx]):
            nbr_idx = neighbors[idx, nbr]
            xji = pos[idx] - pos[nbr_idx]
            currForce += -(pressure[idx] + pressure[nbr_idx]) * 0.5 * m[idx] / density[idx] * gradient_w_spiky(xji)
        force[idx] += currForce

@ti.func
def update_viscosity_force():
    for idx in range(num_particles[None]):
        currForce = tm.vec2(0,0)
        for nbr in range(particle_num_neighbors[idx]):
            nbr_idx = neighbors[idx, nbr]
            xji = pos[idx] - pos[nbr_idx]
            currForce += viscosity_coef * (vel[nbr_idx] - vel[idx]) * m[idx] / density[idx] * laplacian_w_vis(xji)
        force[idx] += currForce

@ti.func
def update_body_force():
    for idx in range(num_particles[None]):
        force[idx] += density[idx] * gravity

@ti.func
def update_boundary_collision_force():
    for i in range(num_particles[None]):
        # boundary check with the bowl
        phi = bowl_r - ((pos[i] - c).norm() * screen_size) - r[i]
        neg_grad = (pos[i] - c).normalized()
        if phi < 0: # outside of the bowl
            force[i] += neg_grad * ks * (phi) * density[i]

@ti.kernel
def substep():
    clear_force()
    update_voxels()
    update_neighbors()    
    update_density()
    update_pressure()
    update_pressure_force()
    update_viscosity_force()
    update_body_force()
    update_boundary_collision_force()
    # update position and velocity
    for idx in range(num_particles[None]):
        vel[idx] = vel[idx] + dt * force[idx] / density[idx]
        pos[idx] = pos[idx] + dt * vel[idx]

In [7]:
### Setup ###
@ti.kernel
def init_particles():
    sqrt = int(ti.sqrt(init_num_particles))
    disp = 0.5/sqrt
    num_particles[None] = init_num_particles
    for idx in range(init_num_particles):
        r[idx] = particle_radius
        vel[idx] = tm.vec2(0, 0)
        force[idx] = tm.vec2(0, 0)
        m[idx] = 1.0
        pos[idx] = tm.vec2(0.25 + disp * ((idx/ sqrt)), 0.25 + disp * (idx % sqrt))           

@ti.kernel
def new_particle_block(pos_x: ti.f32, pos_y: ti.f32):
    """Creates a new particle block to be added to the screen"""
    first_particle_id = num_particles[None]
    sqrt = int(ti.sqrt(particle_block_size))
    disp = float(new_particle_radius*2 + 5) / screen_size
    for count in range(particle_block_size):
        if first_particle_id + count < max_particles:
            new_particle(pos_x + disp * int(count/ sqrt), pos_y + disp * (count % sqrt))

# Add a new set of particles based on mouse click
@ti.func
def new_particle(pos_x: ti.f32, pos_y: ti.f32):
    particle_id = num_particles[None]
    pos[particle_id] = [pos_x, pos_y]
    vel[particle_id] = [0, 0]
    m[particle_id] = new_particle_mass
    r[particle_id] = new_particle_radius
    num_particles[None] += 1

In [9]:
### Driver ###
def main():
    gui = ti.GUI('Particle Fluid', (screen_size, screen_size))
    ctr = 0
    save_pic = False
        
    init_particles()

    if debug:
        print(pos)
        substep()
        print(pos)
        print(particle_num_neighbors)
        print(neighbors)
        print(density)
        print(force)
        
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == ti.GUI.LMB:
                new_particle_block(e.pos[0], e.pos[1])
            elif e.key == 'c':
                save_pic = True
        
        # Move stuff
        if not debug:
            for step in range(substeps):
                substep()
             
        # Draw particles
        X = pos.to_numpy()
        for i in range(num_particles[None]):
            gui.circle(pos=X[i], radius = r[i])
        
        if save_pic:
            filename = f'particle_fluid_vis_{ctr:02d}.png'
            gui.show(filename)
            ctr+=1
            save_pic = False
        else:
            gui.show()
    
if __name__ == '__main__':
    main()